# Load parquet files, process them into train/val tensors, then save them

### We need to do this because IPython + jupyter is complete ass at memory management.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os, gc
import pandas as pd
import polars as pl
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# check that torch is working and sees the GPU
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.4.1+cu121
True
NVIDIA GeForce RTX 3060


In [3]:
PATH = os.getcwd() + '/input/'

METAS = ['date_id', 'time_id', 'symbol_id', 'weight']
FEATURES = [f'feature_{i:02}' for i in range(79)]
RESPONDERS = [f'responder_{i}' for i in range(9)]
RESPONDERS_LAGS = [f'responder_{i}_lag_1' for i in range(9)]
TARGET = 'responder_6'
FEATURES_AND_LAGS=True
ONLY_TARGET_LAGS=True
if FEATURES_AND_LAGS and not ONLY_TARGET_LAGS:
    FEATURES_WORKING = FEATURES + RESPONDERS + RESPONDERS_LAGS
    FEATURES_WORKING.remove(TARGET)
elif FEATURES_AND_LAGS and ONLY_TARGET_LAGS:
    FEATURES_WORKING = FEATURES + ['responder_6_lag_1']
else:
    FEATURES_WORKING = FEATURES
print(FEATURES_WORKING)

USEFUL_COLS = FEATURES_WORKING + [TARGET] + ['weight']

TEST=False

['feature_00', 'feature_01', 'feature_02', 'feature_03', 'feature_04', 'feature_05', 'feature_06', 'feature_07', 'feature_08', 'feature_09', 'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51', 'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56', 'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_61', 'feature_62', 'feature_63', 'feature_64', 'feature_65', 'feature_66', 'feature_67', 'feature_68', 'feature_69', 'feature_70', 'feat

In [4]:
# should test models with only features, features + responders, + lags, etc...
if TEST:
    train_df = pl.scan_parquet(PATH+'training_TEST.parquet') # this parquet has lags
    valid_df = pl.scan_parquet(PATH+'validation_TEST.parquet') # this parquet has lags
    train_save_path = f'./input/trainsets/working_trainset_TEST.pt'
    valid_save_path = f'./input/validsets/working_validset_TEST.pt'
else:
    train_df = pl.scan_parquet(PATH+'training.parquet').select(USEFUL_COLS)
    valid_df = pl.scan_parquet(PATH+'validation.parquet').select(USEFUL_COLS)
    train_save_path = f'./input/trainsets/working_trainset.pt'
    valid_save_path = f'./input/validsets/working_validset.pt'

feature_tags = pl.read_csv(PATH+'features.csv') # no one seems to use this....
gc.collect()

print('Filling...')
train_df = train_df.select([
    pl.col(c).fill_null(pl.col(c).mean()).alias(c) for c in train_df.columns
])

fold_function = 'None'

if fold_function == 'PurgedGroupTimeSeries':
    X = train_df[FEATURES_WORKING].to_numpy()
    y = train_df[TARGET].to_numpy()
    group = train_df['date_id'].to_numpy()
    w = train_df['weight'].to_numpy()
    # Define the PurgedGroupTimeSeriesCV
    cv = PurgedGroupTimeSeriesSplit(
    n_splits=folds,
    max_train_group_size=80,
    group_gap=10,
    max_test_group_size=20
    )
    folds = cv.split(X=X, y=y, groups=group)
elif fold_function == 'KFold': # TODO
    folds = _
elif fold_function == 'None' :
    folds = [([0,0],[0,0])]
    print('Collecting dataframe...')
    train_dfc = train_df.collect().to_pandas()
    train_dfc = train_dfc.astype(np.float16)
    X_tr, y_tr, w_tr = train_dfc[FEATURES_WORKING].values, train_dfc[TARGET].values, train_dfc["weight"].values
else:
    folds = _
print('Generated data variables.') 
gc.collect()
train_dfc = None
gc.collect()
print('Attempted garbage collection.')

trainset = TensorDataset(torch.tensor(X_tr), torch.tensor(y_tr), torch.tensor(w_tr))
torch.save(trainset, save_path)
print('Saved trainset.')

Filling...
Generated data variables.
Attempted garbage collection.
Saved trainset.


In [5]:
valid_df = valid_df.select([
    pl.col(c).fill_null(pl.col(c).mean()).alias(c) for c in valid_df.columns
])
fold_function = 'None'

if fold_function == 'PurgedGroupTimeSeries':
    X = valid_df[FEATURES_WORKING].to_numpy()
    y = valid_df[TARGET].to_numpy()
    group = valid_df['date_id'].to_numpy()
    w = valid_df['weight'].to_numpy()
    # Define the PurgedGroupTimeSeriesCV
    cv = PurgedGroupTimeSeriesSplit(
    n_splits=folds,
    max_valid_group_size=80,
    group_gap=10,
    max_test_group_size=20
    )
    folds = cv.split(X=X, y=y, groups=group)
elif fold_function == 'KFold': # TODO
    folds = _
elif fold_function == 'None' :
    folds = [([0,0],[0,0])]
    print('Collecting dataframe...')
    valid_dfc = valid_df.collect().to_pandas()
    valid_dfc = valid_dfc.astype(np.float16)
    X_vl, y_vl, w_vl = valid_dfc[FEATURES_WORKING].values, valid_dfc[TARGET].values, valid_dfc["weight"].values
else:
    folds = _
print('Generated data variables.') 
gc.collect()
valid_dfc = None
gc.collect()
print('Attempted garbage collection.')

validset = TensorDataset(torch.tensor(X_vl), torch.tensor(y_vl), torch.tensor(w_vl))
torch.save(validset, save_path)
print('Saved validset.')

Generated data variables.
Attempted garbage collection.
Saved validset.
